# Run Sandwich Attacks on Swaps from the Public Mempool

In [1]:
%load_ext autoreload
%autoreload 2

In [12]:
import os
import sys

current_path = sys.path[0]
sys.path.append(
    current_path[: current_path.find("defi-measurement")]
    + "liquidity-distribution-history"
)

sys.path.append("..")


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set display options
pd.set_option('display.max_colwidth', None)  # Display entire cell content
pd.set_option('display.max_rows', 50)    # Display all rows
pd.set_option('display.max_columns', None) # Display all columns


import psycopg2
import sqlalchemy
from dotenv import load_dotenv
from matplotlib.ticker import MaxNLocator
from pool_state import v3Pool
from sqlalchemy import create_engine
from tqdm import tqdm
from collections import deque

load_dotenv(override=True)
from experiments.random_permutations import load_pool

from decimal import getcontext

getcontext().prec = 100  # Set the precision high enough for our purposes


# Read in the environment variables
postgres_uri_mp = os.environ["POSTGRESQL_URI_MP"]
postgres_uri_us = os.environ["POSTGRESQL_URI_US"]

engine = create_engine(postgres_uri_mp)

## Get the Data

In [3]:
query = """
SELECT *
FROM SWAP_LIMIT_PRICE AS LIM
INNER JOIN MEMPOOL_TRANSACTIONS AS MEM ON LIM.CALL_TX_HASH = MEM.HASH
"""

col_rename = dict(
    call_block_number='block_number',
    contract_address='pool',
)

df = pd.read_sql_query(query, engine).rename(columns=col_rename).sort_values(by=['pool', 'block_number'])

df.shape

(828111, 14)

In [4]:
swap_counts = df.groupby(by=['pool', 'block_number'])[['call_success']].count().sort_values('call_success', ascending=False)

swap_counts[swap_counts == 1].call_success.sum(), swap_counts[swap_counts > 1].call_success.sum(), swap_counts.call_success.sum()

(692368.0, 135743.0, 828111)

## Create Sandwich Attacks on Single Swaps

Start with this to validate the approach.

In [21]:
single_swap_blocks = swap_counts[swap_counts == 1].sort_index()

df_single = df.set_index(['pool', 'block_number']).loc[single_swap_blocks.index]

# Group by level 0 and count unique values in level 1
grouped_counts = df_single.groupby(level=0).apply(lambda x: x.index.get_level_values(1).nunique())

# Sort the indices based on the counts
sorted_indices = grouped_counts.sort_values(ascending=False).index

# Reindex the DataFrame based on this sorted order
df_single_sorted = df_single.loc[sorted_indices]

from decimal import Decimal, getcontext

# Set the precision to a sufficiently high value
getcontext().prec = 100

df_single_sorted = df_single_sorted.dropna(subset=['sqrtPriceLimitX96', 'output_amount0', 'output_amount1'])

# Convert the fixed-point numbers to Decimal and perform the calculation
df_single_sorted['priceLimit'] = df_single_sorted['sqrtPriceLimitX96'].apply(lambda x: float((Decimal(x) / (2**96))**2))

# Calculate the price realized per swap
df_single_sorted['priceRealized'] = df_single_sorted['output_amount1'].astype(float).abs() / df_single_sorted['output_amount0'].astype(float).abs()

df_single_sorted[df_single_sorted.zeroForOne].head(n=50)

call_success  \
pool                                       block_number                 
0x11b815efb8f581194ae79006d24e0d814b7697f6 17552207              True   
                                           17552220              True   
                                           17552232              True   
                                           17552233              True   
                                           17552269              True   
                                           17552271              True   
                                           17552279              True   
                                           17552329              True   
                                           17552354              True   
                                           17552555              True   
                                           17552575              True   
                                           17552588              True   
                                           17552596              True   
                                           17552613              True   
                                           17552678              True   
                                           17552686              True   
                                           17552700              True   
                                           17552715              True   
                                           17552762              True   
                                           17552787              True   
                                           17552807              True   
                                           17552828              True   
                                           17552841              True   
                                           17552851              True   
                                           17552898              True   
                                           17552928              True   
                                           17552936              True   
                                           17552953              True   
                                           17552963              True   
                                           17552985              True   
                                           17552997              True   
                                           17553007              True   
                                           17553011              True   
                                           17553017              True   
                                           17553060              True   
                                           17553091              True   
                                           17553123              True   
                                           17553134              True   
                                           17553136              True   
                                           17553139              True   
                                           17553142              True   
                                           17553170              True   
                                           17553172              True   
                                           17553175              True   
                                           17553176              True   
                                           17553187              True   
                                           17553194              True   
                                           17553196              True   
                                           17553209              True   
                                           17553217              True   

                                                                                                               call_tx_hash  \
pool                                       block_number                                                                       
0x11b8

EOFError: Ran out of input

In [25]:
pool = load_pool("0x11b815efb8f581194ae79006d24e0d814b7697f6", postgres_uri_us)


pool.getPriceAt(as_of=17553142.0)**2

Loading pool from database


4.325690314924934e-05

In [26]:
float("4.325690314924934e-05") ** 2

1.8711596700635374e-09

In [7]:
def get_mev_single():
    pass

In [8]:
for pool_addr, blocks in single_swap_blocks.items():
    print(f"Pool: {pool_addr}")
    # pool = load_pool(pool_addr, postgres_uri_us)

    if pool_addr == "0x11b815efb8f581194ae79006d24e0d814b7697f6":
        print("Skipping")
        continue

    for block in blocks:

        print(f"Block: {block}")

        swap = df[(df.pool == pool_addr) & (df.block_number == block)]

        print(swap.iloc[1].hash)
        assert len(swap) == 1, f"Expected 1 swap, got {len(swap)}"

        swap = swap.iloc[0]

        print(swap.hash)
        print(swap)


        swap_params =         {
            "input": int(swap.amountSpecified),
            "tokenIn": pool.token0 if swap.zeroForOne else pool.token1,
            "as_of": swap.block_number,
            "gasFee": True,
        }

        output, heur = pool.swap_in(swap_params, fees=True)

        print(f"Swap ({swap.amountSpecified} for {output})")
        print(f"Next sqrt price: {heur.sqrtP_next} vs. sqrtPriceLimit: {swap.sqrtPriceLimitX96}")
        print(f"Sqrt Price: {heur.sqrt_P}")

        assert False


Pool: call_success
Block: 1.0


IndexError: single positional indexer is out-of-bounds

In [ ]:
swap

,pool,call_success,call_tx_hash,call_trace_address,call_block_time,block_number,amountSpecified,output_amount0,output_amount1,recipient,sqrtPriceLimitX96,zeroForOne,hash,first_seen
25074,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,True,0xa1dc1bc7b54c6a13c48718714cf9524f34621d955b61...,[0],2023-06-24 22:01:59+00:00,17552211,5000000000,5000000000,-2663464990371447655,0xb780f89d37864492a81c7b0053879486d259f16c,4295128740,True,0xa1dc1bc7b54c6a13c48718714cf9524f34621d955b61...,2023-06-24 22:01:48.527000+00:00
98402,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,True,0xa5a2a94332d6c29d167f6bfac54b541ac056203b62b7...,[0 3 3 0 0],2023-06-24 22:01:59+00:00,17552211,452238284,452238284,-240903021710830509,0xdef1c0ded9bec7f1a1670819833240f027b25eff,4295128740,True,0xa5a2a94332d6c29d167f6bfac54b541ac056203b62b7...,2023-06-24 22:01:50.697000+00:00
575369,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,True,0xcccbb87fc3e5e2b7851adc2140f46603c39eebef6480...,[0],2023-06-24 22:01:59+00:00,17552211,3200000000,3200000000,-1704604062672965831,0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad,4295128740,True,0xcccbb87fc3e5e2b7851adc2140f46603c39eebef6480...,2023-06-24 22:01:55.725000+00:00


In [ ]:
from decimal import Decimal, getcontext

# Ensure that we have enough precision
getcontext().prec = 100

# Example sqrtPriceLimitX96 value
sqrt_price_limit_x96_str = "4295128740"
sqrt_price_limit_x96 = Decimal(sqrt_price_limit_x96_str)

# Convert to the square root of the price
sqrt_price = sqrt_price_limit_x96 / (2**96)

# Calculate the actual price
actual_price = sqrt_price * sqrt_price

sqrt_price, actual_price


(Decimal('5.42121463340349417168512463012009817975300762782353558577597141265869140625E-20'),
 Decimal('2.938956810142818170490203137305544242974071560518843705584884307271284930443817505824327569078380388E-39'))